# Tutorial part 4

In [part 3](part3.ipynb) we have learnt about portfolio states. We will continue with that, here.

## Example data

We start with a similar portfolio state as in the previous part:

In [1]:
import portfolyo as pf
import pandas as pd

index = pd.date_range('2024-09-20', '2024-12-10', freq='15T', inclusive='left', tz='Europe/Berlin')
# Creating offtake portfolio line.
ts_offtake = -1 * pf.dev.w_offtake(index, avg=50)
offtake = pf.PfLine({'w': ts_offtake})
# Creating portfolio line with market prices (here: forward price curve).
ts_prices = pf.dev.p_marketprices(index, avg=200)
prices = pf.PfLine({'p': ts_prices})

# Creating portfolio line with sourced volume.
ts_sourced_power1, ts_sourced_price1 = pf.dev.wp_sourced(ts_offtake, 'QS', 0.3, p_avg=120)
sourced_quarters = pf.PfLine({'w': ts_sourced_power1, 'p': ts_sourced_price1})
ts_sourced_power2, ts_sourced_price2 = pf.dev.wp_sourced(ts_offtake, 'MS', 0.2, p_avg=150)
sourced_months = pf.PfLine({'w': ts_sourced_power2, 'p': ts_sourced_price2})
sourced = pf.MultiPfLine({'quarter_products': sourced_quarters, 'month_products': sourced_months})

# Create the portfolio state.
ref = pf.PfState(offtake, prices, sourced)

### Quick recap

The current best-guess procurement price is the following:

In [2]:
ref.asfreq("MS").pnl_cost.p # or ref.pnl_cost.asfreq("MS").p

2024-10-01 00:00:00+02:00    178.84459246623723
2024-11-01 00:00:00+01:00    182.39459192946475
Freq: MS, Name: p, dtype: pint[Eur/MWh]

Our portfolio is only partly hedged...

In [3]:
ref.asfreq("MS").sourcedfraction

2024-10-01 00:00:00+02:00    0.46554768311153116
2024-11-01 00:00:00+01:00     0.5844019411391619
Freq: MS, Name: fraction, dtype: pint[]

... which means that a market price increase has a significant impact on the procurement price:

In [4]:
higherprices = prices + pf.Q_(50, "Eur/MWh")
pfs_higherprices = ref.set_unsourcedprice(higherprices)
pfs_higherprices.asfreq("MS").pnl_cost.p

2024-10-01 00:00:00+02:00    205.56720831066005
2024-11-01 00:00:00+01:00     203.1744948725072
Freq: MS, Name: p, dtype: pint[Eur/MWh]

## Hedging

Hedging can reduce the sensitivity of our portfolio to changes in the market price. Given the current market price curve, we can calculate how much we'd need to source to obtain a fully hedged portfolio:

In [8]:
needed = ref.hedge_of_unsourced("val", "MS")  # value hedge with month products

Let's say we procure exactly that volume. We can add it to the sourced volume in our portfolio state:

In [9]:
hedged = ref.add_sourced(pf.MultiPfLine({"newvolume": needed}))


c:\users\ruud.wijtvliet\ruud\python\dev\portfolyo\portfolyo\core\pfstate\pfstate.py:196: UserWarning: This changes the unsourced volume and causes inaccuracies in its price, if the portfolio has a frequency that is longer than the spot market.
  warnings.warn(


The portfolio is now hedged at the month level. We can verify this by looking at the unsourced volume. In case of a volume hedge, the unsourced volume (``q`` and ``w``) is 0, even if its monetary value is not; in case of a value hedge, it is the reverse:

In [11]:
hedged.unsourced.asfreq('MS')

PfLine object with price and volume information.
. Timestamps: first: 2024-10-01 00:00:00+02:00     timezone: Europe/Berlin
               last: 2024-11-01 00:00:00+01:00         freq: <MonthBegin> (2 datapoints)
. Children: none

                                     w           q           p             r
                                    MW         MWh     Eur/MWh           Eur

2024-10-01 00:00:00 +0200         -0.2        -175       -0.00             0
2024-11-01 00:00:00 +0100         -0.2        -134       -0.00             0

Because the market prices have not changed, the best-estimate procurement prices are also unchanged from before:

In [ ]:
hedged.pnl_cost.p - ref.pnl_cost.p

### Market price change

A hedged profile is less impacted by market price changes. To see that this is indeed the case, let's look at a scenario with an increase in the forward price curve by 40 Eur/MWh, for both portfolio states:

In [10]:
newprices = prices + pf.Q_(40, 'Eur/MWh')
ref_higherprices = ref.set_unsourcedprice(newprices)
hedged_higherprices = hedged.set_unsourcedprice(newprices)

The reference portfolio has gotten a lot more expensive, whereas the procurement price for the hedged portfolio has not moved significantly:

In [13]:
pd.DataFrame({
    ('ref', 'before'): ref.pnl_cost.asfreq('MS').p, 
    ('ref', 'after'): ref_higherprices.pnl_cost.asfreq('MS').p, 
    ('hedged', 'before'): hedged.pnl_cost.asfreq('MS').p, 
    ('hedged', 'after'): hedged_higherprices.pnl_cost.asfreq('MS').p,
}).pint.dequantify()

ref                  hedged            
                               before       after      before       after
unit                          Eur/MWh     Eur/MWh     Eur/MWh     Eur/MWh
2024-10-01 00:00:00+02:00  178.844592  224.127688  178.844592  178.844592
2024-11-01 00:00:00+01:00  182.394592  222.304033  182.394592  182.394592

For the observant reader: it may seem that the portfolio was not fully hedged after all, as a small change in the procurement price is seen. The reason is that each strategy (i.e., volume or value hedge) fully protects only against a specific price change (i.e., absolute or relative). A volume hedge does not *fully* hedge against an absolute price change such as the one we see here.